In [1]:

#SELECT marketRegionId, marketMarketId, marketCollected, category1, category2, marketBrandGoldenApple, marketName, marketDiscountPrice, marketPriceValue, marketUrl,marketAvailability FROM pricing.competitors_price_sephora;

In [2]:
import numpy as np
import math
import pandas as pd

df = pd.read_csv(r'C:\Users\stukova_a\Downloads\competitors_price_sephora_202412311505.txt', 
                delimiter='\t',
                encoding='utf-8')

#pip install pandas openpyxl
# Находим колонки, которые содержат '|'
for column in df.columns:
    if df[column].dtype == 'object':  # Проверяем только текстовые колонки
        if df[column].str.contains('|', regex=False).any():
            # Разбиваем колонку на несколько новых
            split_cols = df[column].str.split('|', expand=True)
            
            # Называем новые колонки
            for i in range(len(split_cols.columns)):
                new_col_name = f"{column}_{i+1}"
                df[new_col_name] = split_cols[i]
            
            # Удаляем исходную колонку
            df = df.drop(columns=[column])
# Словарь для переименования колонок
df.columns  = ["Пусто1",
    "marketRegionId",
    "marketMarketId",
    "marketCollected",
    "category1",
    "category2",
    "marketBrandGoldenApple",
    "marketName",
    "marketDiscountPrice",
    "marketPriceValue",
    "marketUrl",
    "marketAvailability","Пусто2","Пусто3","Пусто4"]

#df
def remove_empty_columns(df):
    # Remove columns where all values are empty/null
    # This includes columns with only None, NaN, empty strings, or "Пусто"
    return df.dropna(axis=1, how='all').drop(columns=df.columns[
        (df == "Пусто1").all() | 
        (df == "Пусто2").all() | 
        (df == "Пусто3").all() | 
        (df == "Пусто4").all() | 

        (df == "").all() | 
        (df.isna().all())
    ])
df = df.drop(0)
def clean_columns(df):
    # Удаляем колонки, в названии которых есть "Пусто"
    clean_cols = [col for col in df.columns if not col.startswith('Пусто1')]
    clean_cols = [col for col in df.columns if not col.startswith('Пусто2')]
    clean_cols = [col for col in df.columns if not col.startswith('Пусто3')]
    clean_cols = [col for col in df.columns if not col.startswith('Пусто4')]

    # Из оставшихся колонок оставляем только те, где есть данные
    df = df[clean_cols]
    df = df.dropna(axis=1, how='all')
    
    return df

def clean_columns(df):
    # Удаляем колонки, которые:
    # 1. Содержат "Пусто" где угодно в названии
    # 2. Начинаются с цифр
    # 3. Состоят только из пустых значений
    
    clean_cols = [col for col in df.columns 
                 if 'Пусто' not in col  # убираем колонки с "Пусто"
                 and not any(c.isdigit() for c in col[0])  # убираем колонки, начинающиеся с цифр
                 and not df[col].isna().all()  # убираем полностью пустые колонки
                 and not (df[col] == '').all()  # убираем колонки с пустыми строками
                 and not (df[col] == 'None').all()]  # убираем колонки со значением None
    
    return df[clean_cols]
needed_columns = [
    'marketRegionId',
    'marketMarketId',
    'marketCollected',
    'category1',
    'category2',
    'marketBrandGoldenApple',
    'marketName',
    'marketDiscountPrice',
    'marketPriceValue',
    'marketUrl',
    'marketAvailability'
]

# Создаем новый датафрейм только с нужными колонками
new_df = df[needed_columns]

df = new_df

# Конвертируем ценовые колонки в числовой формат
df['marketDiscountPrice'] = pd.to_numeric(df['marketDiscountPrice'], errors='coerce')
df['marketPriceValue'] = pd.to_numeric(df['marketPriceValue'], errors='coerce')

# Расчет скидки
def calculate_discount(row):
    if pd.isna(row['marketDiscountPrice']) or pd.isna(row['marketPriceValue']):
        return None
    if row['marketPriceValue'] == 0:  # избегаем деления на ноль
        return 0

    # Определяем большую и меньшую цену
    base_price = max(row['marketPriceValue'], row['marketDiscountPrice'])
    discounted_price = min(row['marketPriceValue'], row['marketDiscountPrice'])

    discount = round(((base_price - discounted_price) / base_price) * 100, 2)

    # Если скидка 100%, значит данные некорректные
    if discount == 100:
        return 0
    return discount

# Добавляем колонку со скидкой и исправляем цены
df['discount_percent'] = df.apply(calculate_discount, axis=1)

# Где скидка была 100%, приравниваем цену после скидки к цене до скидки
mask = (df['discount_percent'] == 0) & (df['marketDiscountPrice'] != df['marketPriceValue'])
df.loc[mask, 'marketDiscountPrice'] = df.loc[mask, 'marketPriceValue']

# Проверяем среднюю скидку на наличие NaN и заменяем на 0, если они есть
average_discount = df['discount_percent'].mean()
if pd.isna(average_discount):
    average_discount = 0

# Вывод исправленных результатов
print("\nПримеры расчета скидок:")
print(df[['marketName', 'marketPriceValue', 'marketDiscountPrice', 'discount_percent']].head(10))

print("\nСтатистика по скидкам:")
print(f"Количество товаров со скидкой: {len(df[df['discount_percent'] > 0])}")
print(f"Средняя скидка: {average_discount:.2f}%")
print(f"Максимальная скидка: {df['discount_percent'].max():.2f}%")

# Проверка на 100% скидки
print("\nПроверка на наличие 100% скидок:")
hundred_percent = df[df['discount_percent'] == 100]
print(f"Количество товаров со 100% скидкой: {len(hundred_percent)}")

df['discount_percent'] = df['discount_percent'].replace([np.nan, None, 'NaN', 'None'], 0.0)


# Замена некорректных значений в discount_percent
df['discount_percent'] = df['discount_percent'].replace([np.nan, None, 'NaN', 'None'], 0.0)

# Очистка marketCollected перед преобразованием
# Удаляем лишние пробелы, заменяем некорректные значения на NaT
df['marketCollected'] = df['marketCollected'].str.strip()  # Удаляем лишние пробелы
df['marketCollected'] = pd.to_datetime(
    df['marketCollected'], errors='coerce', format='%Y-%m-%d %H:%M:%S'
)  # Преобразуем в дату, некорректные значения заменяются на NaT

# Удаляем время, оставляя только дату
df['marketCollected'] = df['marketCollected'].dt.date

# Проверка результата
print(df['marketCollected'].head())

# Удаляем пробелы и специальные символы
def clean_string(x):
    if isinstance(x, str):
        return x.strip().replace('¶', ' ')
    return x

# Применяем очистку только к строковым колонкам
df = df.apply(lambda x: x.map(clean_string) if x.dtype == "object" else x)


unique_columns = [
    "marketRegionId",
    "marketName",
    
    "category1",
    "marketCollected", "marketUrl"
]
df_unique = df.drop_duplicates(subset=unique_columns)

#df_unique

df = df_unique[~df_unique['marketAvailability'].str.contains('\d', regex=True, na=False)]
#df.head(10000).to_excel(r'C:\Users\stukova_a\Downloads\first_10_rows.xlsx', index=False)
#df

# Преобразуем marketCollected в datetime, если это ещё не сделано
df['marketCollected'] = pd.to_datetime(df['marketCollected'])

# Добавляем столбец с неделей года
df['week_of_year'] = df['marketCollected'].dt.isocalendar().week

# Добавляем столбец с названием месяца
df['month_name'] = df['marketCollected'].dt.strftime('%B')




Примеры расчета скидок:
                                           marketName  marketPriceValue  \
1   The Scent Elixir - Parfum Intense - 50ml      ...             470.0   
2   The Scent Elixir - Parfum Intense - 100ml     ...             650.0   
3   Jazz Club - Eau De Toilette - 30ml            ...             334.0   
4   Jazz Club - Eau De Toilette - Eau de Toilette ...             777.0   
5   Replica From The Garden - 100ml               ...             777.0   
6   Replica From The Garden - 30ml                ...             334.0   
7   The Scent Elixir - Parfum Intense - 50ml      ...             635.0   
8   Libre - Eau de Parfum - 30ml                  ...             426.0   
9   Libre - Eau de Parfum - 50ml                  ...             587.0   
10  Libre - Eau de Parfum - 90ml                  ...             777.0   

    marketDiscountPrice  discount_percent  
1                 470.0               0.0  
2                 650.0               0.0  
3                

C:\Users\Stukova_a\AppData\Local\Temp\ipykernel_9692\1844157606.py:191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['marketCollected'] = pd.to_datetime(df['marketCollected'])
C:\Users\Stukova_a\AppData\Local\Temp\ipykernel_9692\1844157606.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['week_of_year'] = df['marketCollected'].dt.isocalendar().week
C:\Users\Stukova_a\AppData\Local\Temp\ipykernel_9692\1844157606.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [3]:
#df

In [5]:
# Create availability summary table
def create_availability_summary(df):
    # Ensure proper data types
    df['marketCollected'] = pd.to_datetime(df['marketCollected'])
    
    # Create separate dataframes for InStock and OutOfStock
    df_instock = df[df['marketAvailability'].str.contains('InStock', case=False, na=False)]
    df_outstock = df[df['marketAvailability'].str.contains('OutOfStock', case=False,na=False)]
    # Function to calculate metrics for each availability status
    def calculate_metrics(data):
        # Count total items
        total_count = len(data)
        
        # Calculate average discount only for items with discount > 0
        items_with_discount = data[data['discount_percent'] > 0]
        avg_discount = items_with_discount['discount_percent'].mean()
        discount_count = len(items_with_discount)
        
        return total_count, avg_discount, discount_count
    
    # Group by date and region for both InStock and OutOfStock
    results = []
    
    for date, region in df.groupby(['marketCollected', 'marketRegionId']):
        instock_data = df_instock[(df_instock['marketCollected'] == date[0]) & 
                                (df_instock['marketRegionId'] == date[1])]
        outstock_data = df_outstock[(df_outstock['marketCollected'] == date[0]) & 
                                  (df_outstock['marketRegionId'] == date[1])]
        
        instock_count, instock_avg_disc, instock_disc_count = calculate_metrics(instock_data)
        outstock_count, outstock_avg_disc, outstock_disc_count = calculate_metrics(outstock_data)
        
        results.append({
            'marketRegionId': date[1],
            'marketCollected': date[0],
            'week_of_year': date[0].isocalendar().week,
            'month_name': date[0].strftime('%B'),
            'InStocks_Count': instock_count,
            'InStocks_Avg_Discount': round(instock_avg_disc if pd.notnull(instock_avg_disc) else 0),
            'InStocks_Discount_Count': instock_disc_count,
            'OutOfStocks_Count': outstock_count,
            'OutOfStocks_Avg_Discount': round(outstock_avg_disc if pd.notnull(outstock_avg_disc) else 0),
            'OutOfStocks_Discount_Count': outstock_disc_count
        })
    
    result_df = pd.DataFrame(results)
    
    # Format the date
    result_df['marketCollected'] = result_df['marketCollected'].dt.strftime('%d.%m.%Y')
    
    # Sort by region and date
    result_df = result_df.sort_values(['marketRegionId', 'marketCollected'])
    
    # Rename columns to match desired format
    result_df.columns = [
        'marketRegionId', 'marketCollected', 'week_of_year', 'month_name',
        'InStocks', 'InStocks Avg_Discount', 'InStocks Discount_Count',
        'OutOfStocks', 'OutOfStocks Avg_Discount', 'OutOfStocks Discount_Count'
    ]
    
    return result_df

# Apply the function to your dataframe
result_df = create_availability_summary(df)

# Save to Excel if needed
result_df.to_excel('availability_summary.xlsx', index=False)

# Display the first few rows
print(result_df.head())

C:\Users\Stukova_a\AppData\Local\Temp\ipykernel_9692\3112578834.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['marketCollected'] = pd.to_datetime(df['marketCollected'])


    marketRegionId marketCollected  week_of_year month_name  InStocks  \
47           Дубай      01.06.2024            22       June      8949   
100          Дубай      01.07.2024            27       July      8949   
159          Дубай      01.08.2024            31     August     10067   
244          Дубай      01.09.2024            35  September     10838   
326          Дубай      01.10.2024            40    October     11364   

     InStocks Avg_Discount  InStocks Discount_Count  OutOfStocks  \
47                      39                      392         1288   
100                     39                      393         1288   
159                      0                        0          878   
244                      0                        0          874   
326                      0                        0          708   

     OutOfStocks Avg_Discount  OutOfStocks Discount_Count  
47                         40                         472  
100                        40   

In [6]:
result_df.to_excel(r'C:\Users\stukova_a\Downloads\summary_report_SephoraTest25.xlsx', engine='openpyxl') 

In [7]:
def create_weekly_availability_summary(df):
    # Convert marketCollected to datetime if needed
    df['marketCollected'] = pd.to_datetime(df['marketCollected'])
    
    # Add week start and end dates
    df['week_start'] = df['marketCollected'] - pd.to_timedelta(df['marketCollected'].dt.dayofweek, unit='D')
    df['week_end'] = df['week_start'] + pd.Timedelta(days=6)
    
    def get_month_range(start_date, end_date):
        start_month = start_date.strftime('%B')
        end_month = end_date.strftime('%B')
        return start_month if start_month == end_month else f"{start_month}-{end_month}"
    
    results = []
    
    # Group by week, region, category, and product
    for (week_start, region, category), week_group in df.groupby(['week_start', 'marketRegionId', 'category1']):
        week_end = week_start + pd.Timedelta(days=6)
        week_number = week_start.isocalendar().week
        month_range = get_month_range(week_start, week_end)
        
        # Group by product within the week
        product_stats = []
        for product, product_group in week_group.groupby('marketName'):
            # Check if product was InStock at least once during the week
            was_instock = product_group['marketAvailability'].str.contains('InStock', case=False, na=False).any()
            
            # Get the status with priority to InStock
            if was_instock:
                status = 'InStock'
                status_group = product_group[product_group['marketAvailability'].str.contains('InStock', case=False, na=False)]
            else:
                status = 'OutOfStock'
                status_group = product_group[product_group['marketAvailability'].str.contains('OutOfStock', case=False, na=False)]
            
            # Calculate average discount for the product
            avg_discount = status_group['discount_percent'].mean()
            has_discount = avg_discount > 0 if pd.notnull(avg_discount) else False
            
            product_stats.append({
                'status': status,
                'avg_discount': avg_discount if pd.notnull(avg_discount) else 0,
                'has_discount': has_discount
            })
        
        # Calculate daily averages
        total_products = len(product_stats)
        instock_count = sum(1 for p in product_stats if p['status'] == 'InStock')
        outstock_count = total_products - instock_count
        
        # Calculate average discounts
        instock_discounts = [p['avg_discount'] for p in product_stats if p['status'] == 'InStock' and p['has_discount']]
        outstock_discounts = [p['avg_discount'] for p in product_stats if p['status'] == 'OutOfStock' and p['has_discount']]
        
        results.append({
            'marketRegionId': region,
            'week_period': f"{week_start.strftime('%d.%m.%Y')}-{week_end.strftime('%d.%m.%Y')}",
            'week_of_year': week_number,
            'month_name': month_range,
            'category1': category,
            'InStocks': instock_count,
            'InStocks_Avg_Discount': round(sum(instock_discounts) / len(instock_discounts) if instock_discounts else 0),
            'InStocks_Discount_Count': len(instock_discounts),
            'OutOfStocks': outstock_count,
            'OutOfStocks_Avg_Discount': round(sum(outstock_discounts) / len(outstock_discounts) if outstock_discounts else 0),
            'OutOfStocks_Discount_Count': len(outstock_discounts)
        })
    
    result_df = pd.DataFrame(results)
    result_df = result_df.sort_values(['marketRegionId', 'week_of_year', 'category1'])
    
    # Rename columns
    result_df.columns = [
        'marketRegionId', 'week_period', 'week_of_year', 'month_name', 'category1',
        'InStocks', 'InStocks Avg_Discount', 'InStocks Discount_Count',
        'OutOfStocks', 'OutOfStocks Avg_Discount', 'OutOfStocks Discount_Count'
    ]
    
    return result_df

# Apply the function
result_df3 = create_weekly_availability_summary(df)
result_df3.to_excel('availability_summary_by_week.xlsx', index=False)

C:\Users\Stukova_a\AppData\Local\Temp\ipykernel_9692\1209991438.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['marketCollected'] = pd.to_datetime(df['marketCollected'])
C:\Users\Stukova_a\AppData\Local\Temp\ipykernel_9692\1209991438.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['week_start'] = df['marketCollected'] - pd.to_timedelta(df['marketCollected'].dt.dayofweek, unit='D')
C:\Users\Stukova_a\AppData\Local\Temp\ipykernel_9692\1209991438.py:7: SettingWithCopyWarning: 
A value is trying 

In [8]:
result_df3.to_excel(r'C:\Users\stukova_a\Downloads\summary_report_SephoraTest26.xlsx', engine='openpyxl') 